In [15]:
import alpaca_trade_api as tradeapi
import time
import threading
import csv
import pandas as pd
from datetime import datetime, timedelta

# Alpaca API credentials
API_KEY = 'PKL0QPAXI2XEQN42OD2S'
API_SECRET = 'nkqziIUvRn7rkEihNRVpw9IcNYftpDxVHPRQnyS3'
BASE_URL = 'https://paper-api.alpaca.markets'

# Initialize API
api = tradeapi.REST(API_KEY, API_SECRET, base_url=BASE_URL)

# Main csv file
csv_file = "financial_news.csv"

# News data storage
all_news = []

# List of tickers
tickers = [
    "AAPL", "MSFT", "GOOGL", "AMZN", "FB", "INTC", "TSLA", "NVDA", "ORCL", "IBM",
    "JPM", "BAC", "WFC", "C", "GS", "AXP", "MS", "BRK.B",
    "JNJ", "PFE", "UNH", "MRK", "ABBV", "TMO", "ABT",
    "PG", "KO", "PEP", "NKE", "MO",
    "GE", "MMM", "BA", "HON", "CAT",
    "XOM", "CVX", "BP", "TOT",
    "NEP", "DUK", "SO",
    "T", "VZ",
    "DIS", "MCD", "SBUX",
    "WMT", "HD"
]

Data Collection

In [2]:
# Function to fetch news for a ticker
def fetch_news(ticker):
    global all_news
    try:
        news = api.get_news(symbol=ticker, limit=200)  # Adjust the limit as needed
        
        for article in news:
            all_news.append({
                'headline': article.headline,
                'summary': article.summary,
                'created_at': article.created_at,
                'ticker': ticker
            })
    except Exception as e:
        print(f"Error fetching news for {ticker}: {e}")

# Function to process requests in batches
def process_requests(rate_limit=200):
    while tickers:
        threads = []
        # 200 parallel threads, respecting the rate limit
        for _ in range(rate_limit):  
            if tickers:
                ticker = tickers.pop(0)
                thread = threading.Thread(target=fetch_news, args=(ticker,))
                threads.append(thread)
                thread.start()

        # Wait for all threads to complete
        for thread in threads:
            thread.join()

        # Wait for 60 seconds before the next batch
        time.sleep(60)
    
# Write data to CSV
def write_to_csv():
    with open(csv_file, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.DictWriter(file, fieldnames=all_news[0].keys())
        writer.writeheader()
        for news_item in all_news:
            writer.writerow(news_item)

    print(f'Data saved to {csv_file}')

Data Cleaning Functions

In [3]:
import re

# Keep digits and symbols that can be significant in financial analysis. 
# E.g. digits could represent financial figures, and symbols like '$' might indicate currency amounts. 
def clean_text(text):
    # Remove numbers
    text = re.sub(r'\d+', '', text)  
    # Remove URLs
    text = re.sub(r'http\S+', '', text)  
    # Remove newlines and tabs
    text = text.replace('\n', ' ').replace('\r', '').replace('\t', ' ') 
    # Remove puncutations
    text = re.sub(r'[^\w\s]', '', text)
    # Replace multiple spaces with a single space
    text = re.sub(r'\s+', ' ', text)  
    return text.strip()

In [4]:
'''
Testing clean_data()
'''
# Example DataFrame
data = pd.DataFrame({
    'article': ["Stocks rallied, with AAPL reaching $150.50. More at http://example.com", 
                "Q1 profits fell to $1.5 billion, down from $2 billion. Visit http://finance.com for more.", 
                "Tech stocks, e.g., MSFT, AMZN, and GOOG, showed mixed results today."]
})

data['cleaned_article'] = data['article'].apply(clean_text)
print(data)

                                             article  \
0  Stocks rallied, with AAPL reaching $150.50. Mo...   
1  Q1 profits fell to $1.5 billion, down from $2 ...   
2  Tech stocks, e.g., MSFT, AMZN, and GOOG, showe...   

                                     cleaned_article  
0          Stocks rallied with AAPL reaching More at  
1  Q profits fell to billion down from billion Vi...  
2  Tech stocks eg MSFT AMZN and GOOG showed mixed...  


In [5]:
from datetime import datetime
def extract_date(date_str):
    try:
        # Parse the string to a datetime object
        dt = datetime.strptime(date_str, '%Y-%m-%d %H:%M:%S+00:00')
        # Format the datetime object to keep only the date part
        return dt.strftime('%Y-%m-%d')
    except ValueError as e:
        print(f"Error parsing date: {e}")
        return None

In [6]:
'''
Testing extract_date()
'''
date_string = "2024-01-25 13:37:00+00:00"
extracted_date = extract_date(date_string)
print(extracted_date)

2024-01-25


In [7]:
def fetch_stock_prices(ticker_date_tuples, rate_limit=200, requests_per_minute=200):
    # Ensuring that the API rate limit of 200 per minute is respected
    time.sleep(60)
    
    stock_prices = {}
    request_count = 0

    for ticker, date in ticker_date_tuples:
        try:
            bars = api.get_bars(ticker, '1Day', start=date, end=date)

            if bars:
                # Access the closing price with .c
                closing_price = bars[0].c  
                stock_prices[(ticker, date)] = closing_price
#                     print(ticker)
#                     print(date)
#                     print(closing_price)
            else:
                # No data for this date
                stock_prices[(ticker, date)] = None  

            request_count += 1
            if request_count >= rate_limit:
                time.sleep(60)
                request_count = 0

        except Exception as e:
            print(f"Error fetching data for {ticker} on {date}: {e}")
            stock_prices[(ticker, date)] = None
            break

    return stock_prices

In [8]:
# # Gather data and write to csv file
# process_requests()
# write_to_csv()

Data Cleaning 

In [28]:
df = pd.read_csv(csv_file)
df = df.dropna().reset_index(drop=True)

df['headline_summary'] = df['headline'] + " " + df['summary']
df['cleaned_text'] = df['headline_summary'].apply(clean_text)
df['cleaned_date'] = df['created_at'].apply(extract_date)

# Store unique (ticker, date) for more efficient fetching of historical stock prices
ticker_date_tuples = list(df[['ticker', 'cleaned_date']].drop_duplicates().itertuples(index=False, name=None))
# historical_stock_prices = fetch_stock_prices(ticker_date_tuples)
# Save the historical prices into df
df['historical_price'] = df.apply(lambda row: historical_stock_prices.get((row['ticker'], row['cleaned_date'])), axis=1)

yesterday_date = (datetime.now() - timedelta(days=1)).strftime('%Y-%m-%d')
# current_stock_prices = fetch_stock_prices([(ticker, yesterday_date) for ticker in tickers])

# Save the current prices into df
df['latest_price'] = df.apply(lambda row: current_stock_prices.get((row['ticker'], yesterday_date)), axis=1)

# Drop rows with null values
df = df.dropna().reset_index(drop=True)
df

,headline,summary,created_at,ticker,headline_summary,cleaned_text,cleaned_date,historical_price,latest_price
0,How We Missed A Winning Trade On IBM--And How ...,We Won&#39;t Make This Mistake Again\n\n,2024-01-26 15:58:15+00:00,IBM,How We Missed A Winning Trade On IBM--And How ...,How We Missed A Winning Trade On IBMAnd How We...,2024-01-26,187.42,186.90
1,Investigating IBM's Standing In IT Services In...,,2024-01-26 16:01:45+00:00,IBM,Investigating IBM's Standing In IT Services In...,Investigating IBMs Standing In IT Services Ind...,2024-01-26,187.42,186.90
2,90's Tech Titans Are Outshining 'Magnificent S...,Companies that have strong cash flows will lik...,2024-01-26 12:12:06+00:00,IBM,90's Tech Titans Are Outshining 'Magnificent S...,s Tech Titans Are Outshining Magnificent Seven...,2024-01-26,187.42,186.90
3,Why Humana Shares Are Trading Lower By Over 11...,Shares of Humana Inc. (NYSE: HUM) fell sharply...,2024-01-25 19:52:24+00:00,IBM,Why Humana Shares Are Trading Lower By Over 11...,Why Humana Shares Are Trading Lower By Over He...,2024-01-25,190.43,186.90
4,"S&P 500 Eyes Sixth Green Session, Tesla Tumble...",U.S. stock market gains due to strong economic...,2024-01-25 19:22:36+00:00,IBM,"S&P 500 Eyes Sixth Green Session, Tesla Tumble...",SP Eyes Sixth Green Session Tesla Tumbles Oil ...,2024-01-25,190.43,186.90
...,...,...,...,...,...,...,...,...,...
6246,You Can Have Any Color Tesla Model X To Qualif...,While stealth grey is included in the base pri...,2024-01-18 09:47:05+00:00,TSLA,You Can Have Any Color Tesla Model X To Qualif...,You Can Have Any Color Tesla Model X To Qualif...,2024-01-18,211.88,188.86
6247,Tesla Invites Chinese Fans To Decide Cybertruc...,"In China, pickup trucks are classified as ligh...",2024-01-18 08:50:06+00:00,TSLA,Tesla Invites Chinese Fans To Decide Cybertruc...,Tesla Invites Chinese Fans To Decide Cybertruc...,2024-01-18,211.88,188.86
6248,Tesla Loses Most-Shorted Crown To This 'Magnif...,Apple and Microsoft have around $18.63 billion...,2024-01-18 07:38:57+00:00,TSLA,Tesla Loses Most-Shorted Crown To This 'Magnif...,Tesla Loses MostShorted Crown To This Magnific...,2024-01-18,211.88,188.86
6249,China's Top Auto Makers Huddle In Beijing To P...,Early signs of a slowdown in the Chinese auto ...,2024-01-18 06:54:29+00:00,TSLA,China's Top Auto Makers Huddle In Beijing To P...,Chinas Top Auto Makers Huddle In Beijing To Pl...,2024-01-18,211.88,188.86


In [29]:
from nltk.tokenize import word_tokenize
import nltk
# nltk.download('punkt')

# Tokenisation
df['tokens'] = df['cleaned_text'].apply(word_tokenize)

In [30]:
# Removing Stop Words
from nltk.corpus import stopwords
# nltk.download('stopwords')

stop_words = set(stopwords.words('english'))

df['filtered_tokens'] = df['tokens'].apply(lambda tokens: [word for word in tokens if word not in stop_words])

In [33]:
# Lemmatization
from nltk.stem import WordNetLemmatizer
# nltk.download('wordnet')

lemmatizer = WordNetLemmatizer()
df['lemmatized'] = df['filtered_tokens'].apply(lambda tokens: [lemmatizer.lemmatize(word) for word in tokens])

In [34]:
# Save to csv
df.to_csv('financial_news_cleaned.csv', index=False)